In [1]:
import requests

### Entry point for docker/flask on Windows
base = 'http://192.168.99.100:5000'

### Entry point for linux probably
# base = 'http://127.0.0.1:5000'

resp = requests.get(base)
resp

<Response [200]>

In [2]:
# GET /environments
resp = requests.get(base + '/environments')
resp

<Response [200]>

In [3]:
resp.json()

[{'name': 'default', 'url': 'http://localhost:8888'}]

In [4]:
# CREATE new environment
d = {'name' : 'new env',
     'url' : 'https://localhost:1234'}
url = base + '/environments'
resp = requests.post(url, json=d)
resp

<Response [200]>

In [5]:
resp.text

''

In [6]:
# Check that new environment is there
resp = requests.get(base + '/environments')
resp.json()

[{'name': 'default', 'url': 'http://localhost:8888'},
 {'name': 'new env', 'url': 'https://localhost:1234'}]

In [7]:
# GET /notebooks/<id 1>
import uuid
i = uuid.UUID(int=1).hex
resp = requests.get(base + '/notebooks/%s' % i)
resp

<Response [200]>

In [8]:
resp.json()

{'description': 'Default DB notebook entry',
 'notebook': {'cells': [],
  'language_info': {},
  'metadata': {'display_name': 'Python 3',
   'language': 'python',
   'name': 'python3'},
  'nbformat': 4,
  'nbformat_minor': 2},
 'title': 'Default DB notebook',
 'uuid': '00000000000000000000000000000001'}

In [9]:
# CREATE new notebook
import nbformat
nb = nbformat.read('API test notebook.ipynb', as_version=nbformat.NO_CONVERT)
# simulate clear output
for item in nb['cells']:
    if item['cell_type'] == 'code':
        item['outputs'] = []
        
js = {'title' : 'uploaded.ipynb',
      'description' : 'API test notebook upload',
      'notebook' : nb}

url = base + '/stages'
resp = requests.post(url, json=js)
resp

<Response [200]>

In [10]:
new_uuid = resp.text
new_uuid

'a5ed8f92812b4b23b11ec046470357ac'

In [11]:
# GET /notebooks/<new uuid>
import uuid
resp = requests.get(base + '/notebooks/%s' % new_uuid)
resp

<Response [200]>

In [12]:
js = resp.json()
js.keys()

dict_keys(['title', 'description', 'notebook', 'uuid'])

In [13]:
# Write notebook to file
import json
with open(js['title'], 'w') as fp:
    # add nbgallery uuid to notebook metadata
    js['notebook']['metadata']['NBGALLERY_UUID'] = js['uuid']
    s = json.dumps(js['notebook']) # nbformat takes strings to read, not dict objects
    nb = nbformat.reads(s, as_version=nbformat.NO_CONVERT)
    nbformat.write(nb, fp)